# Advanced Document Scanner (Notebook Version)

This notebook demonstrates **each image processing method step-by-step**  
Every method includes its **working name and purpose** for easy understanding.

---
### Libraries Used
- OpenCV
- NumPy
- Matplotlib
- PIL

In [ ]:
# IMPORT REQUIRED LIBRARIES
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


## 1. Point Ordering Method  
**Method Name:** `order_points`  
**Purpose:**  
Reorders detected contour points into a fixed order:
- Top-Left
- Top-Right
- Bottom-Right
- Bottom-Left  
Required for correct perspective transformation.

In [ ]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]      # Top-left
    rect[2] = pts[np.argmax(s)]      # Bottom-right
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]   # Top-right
    rect[3] = pts[np.argmax(diff)]   # Bottom-left
    return rect


## 2. Perspective Transformation  
**Method Name:** `four_point_transform`  
**Purpose:**  
- Removes camera angle distortion  
- Converts document to top-down scanned view

In [ ]:
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = int(max(widthA, widthB))

    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = int(max(heightA, heightB))

    if maxWidth < 50 or maxHeight < 50:
        return None

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (maxWidth, maxHeight))


## 3. Document Detection  
**Method Name:** `detect_document_advanced`  

### Processing Pipeline:
1. Image Resize  
2. Grayscale Conversion  
3. Gaussian Blur (Noise Reduction)  
4. Canny Edge Detection  
5. Morphological Closing  
6. Contour Detection  
7. Polygon Approximation (4 edges)

In [ ]:
def detect_document_advanced(image):
    orig = image.copy()
    ratio = image.shape[0] / 800.0

    resized = cv2.resize(image, (int(image.shape[1]/ratio), 800))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

    # Noise reduction
    blurred = cv2.GaussianBlur(gray, (5,5), 0)

    # Edge detection
    edged = cv2.Canny(blurred, 50, 150)

    # Morphological close
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)

    # Find contours
    cnts, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    for c in cnts[:10]:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            return orig, approx.reshape(4,2) * ratio, True

    return orig, None, False


## 4. Document Enhancement  
**Method Name:** `enhance_document`  

### Enhancement Steps:
1. Grayscale Conversion  
2. Non-Local Means Denoising  
3. CLAHE Contrast Enhancement  
4. Sharpening Filter  
5. Adaptive Thresholding

In [ ]:
def enhance_document(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Noise removal
    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)

    # Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    contrasted = clahe.apply(denoised)

    # Sharpening
    kernel = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
    sharpened = cv2.filter2D(contrasted, -1, kernel)

    # Binarization
    return cv2.adaptiveThreshold(
        sharpened, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 21, 10
    )


## 5. Run Complete Pipeline  
Replace `sample.jpg` with your image file name.

In [ ]:
# Load input image
image = cv2.imread("sample.jpg")

plt.figure(figsize=(6,6))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis("off")


In [ ]:
orig, contour, found = detect_document_advanced(image)

if found:
    warped = four_point_transform(orig, contour)
    scanned = enhance_document(warped)

    plt.figure(figsize=(6,6))
    plt.imshow(scanned, cmap="gray")
    plt.title("Final Scanned Document")
    plt.axis("off")
else:
    print("Document not detected")